In [1]:
import torch
from torch.functional import F
from torch import nn, optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from PIL import Image
from matplotlib import pyplot as plt

/Users/aronsajanphilip/opt/anaconda3/envs/machine_learning/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
activity_map = {
    0: 'safe driving',
    1: 'texting - right',
    2: 'talking on the phone - right',
    3: 'texting - left',
    4: 'talking on the phone - left',
    5: 'operating the radio',
    6: 'drinking',
    7: 'reaching behind',
    8: 'hair and makeup',
    9: 'talking to passenger'
}

reverse_activity_map = {label: cls for cls, label in activity_map.items()}

class DrivingDataset(Dataset):
    def __init__(self, meta_info_dframe, basepath):
        self.info_dframe = meta_info_dframe
        self.basepath = basepath
        self.img_transforms = transforms.Compose([transforms.Resize((200, 200)), 
                                                  transforms.ToTensor(), 
                                                  transforms.Normalize((0.485, 0.456, 0.406), 
                                                                       (0.229, 0.224, 0.225))
                                                 ])
    
    def __len__(self):
        return self.info_dframe.shape[0]
    
    def __getitem__(self, idx):
        img_file = self.info_dframe.iloc[idx]['img']
        label = self.info_dframe.iloc[idx]['label']
        cls = self.info_dframe.iloc[idx]['classname']
        img_file = os.path.join(self.basepath, cls, img_file)
        img = Image.open(img_file)
        img_tensor = self.img_transforms(img)
        return img_tensor, int(label)

def get_image(img_tensor):
    img_arr = img_tensor.numpy()
    img_arr = np.transpose(img_arr, (1,2,0))
    img_arr = img_arr*np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    return img_arr

def show_image_batch(img_tensors, labels):
    fig = plt.figure(figsize=(35,45))
    for i in range(len(img_tensors)):
        img_tensor = img_tensors[i]
        img_arr = get_image(img_tensor)
        ax = fig.add_subplot(10,10,i+1)
        label = labels[i].item()
        ax.set_title(activity_map[label])
        ax.imshow(img_arr)
    plt.show()
    
def show_image_batch(img_tensors, labels, predictions):
    fig = plt.figure(figsize=(45,55))
    for i in range(len(img_tensors)):
        img_tensor = img_tensors[i]
        img_arr = get_image(img_tensor)
        ax = fig.add_subplot(10,10,i+1)
        label = labels[i].item()
        prediction = predictions[i].item()
        ax.set_title("{} | {}".format(activity_map[label], activity_map[prediction]))
        ax.imshow(img_arr)
    plt.show()

In [3]:
info_frame = pd.read_csv("./driver_imgs_list.csv", index_col=False)
info_frame['label'] = info_frame['classname'].apply(lambda x: x[1])

In [4]:
driving_set = DrivingDataset(meta_info_dframe = info_frame, 
                             basepath="/Volumes/DataVolA/ML Datasets/state-farm-distracted-driver-detection/imgs/train")
train_loader = DataLoader(driving_set, shuffle=True, batch_size=100)

In [ ]:
imgs, labels = iter(train_loader).next()

In [ ]:
show_image_batch(imgs, labels)

In [6]:
device = torch.device("mps:0")
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1, device=device)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, device=device)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, device=device)
        self.fc1 = nn.Linear(in_features=25*25*64, out_features=500, device=device)
        self.fc2 = nn.Linear(in_features=500, out_features=10, device=device)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, img_tensor_batch):
        x = F.relu(self.conv1(img_tensor_batch))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        result = x.view(-1,25*25*64)
        result = self.dropout(result)
        result = self.fc1(result)
        result = self.dropout(result)
        result = self.fc2(result)
        return result

In [7]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.03)

In [7]:
EPOCHS = 3

for epoch in range(EPOCHS):
    loss_val = 0
    batch = 0
    for imgs, labels in iter(train_loader):
        batch+=1
        optimizer.zero_grad()
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        predicted = net(imgs)
        loss = criterion(predicted, labels)
        loss.backward()
        loss_val+=loss.item()
        optimizer.step()
        print("Completed Batch {}".format(batch))
    print("Loss: {}".format(loss_val))

Completed Batch 1
Completed Batch 2
Completed Batch 3
Completed Batch 4
Completed Batch 5
Completed Batch 6
Completed Batch 7
Completed Batch 8
Completed Batch 9
Completed Batch 10
Completed Batch 11
Completed Batch 12
Completed Batch 13
Completed Batch 14
Completed Batch 15
Completed Batch 16
Completed Batch 17
Completed Batch 18
Completed Batch 19
Completed Batch 20
Completed Batch 21
Completed Batch 22
Completed Batch 23
Completed Batch 24
Completed Batch 25
Completed Batch 26
Completed Batch 27
Completed Batch 28
Completed Batch 29
Completed Batch 30
Completed Batch 31
Completed Batch 32
Completed Batch 33
Completed Batch 34
Completed Batch 35
Completed Batch 36
Completed Batch 37
Completed Batch 38
Completed Batch 39
Completed Batch 40
Completed Batch 41
Completed Batch 42
Completed Batch 43
Completed Batch 44
Completed Batch 45
Completed Batch 46
Completed Batch 47
Completed Batch 48
Completed Batch 49
Completed Batch 50
Completed Batch 51
Completed Batch 52
Completed Batch 53
Co

Completed Batch 196
Completed Batch 197
Completed Batch 198
Completed Batch 199
Completed Batch 200
Completed Batch 201
Completed Batch 202
Completed Batch 203
Completed Batch 204
Completed Batch 205
Completed Batch 206
Completed Batch 207
Completed Batch 208
Completed Batch 209
Completed Batch 210
Completed Batch 211
Completed Batch 212
Completed Batch 213
Completed Batch 214
Completed Batch 215
Completed Batch 216
Completed Batch 217
Completed Batch 218
Completed Batch 219
Completed Batch 220
Completed Batch 221
Completed Batch 222
Completed Batch 223
Completed Batch 224
Completed Batch 225
Loss: 62.473036501556635
Completed Batch 1
Completed Batch 2
Completed Batch 3
Completed Batch 4
Completed Batch 5
Completed Batch 6
Completed Batch 7
Completed Batch 8
Completed Batch 9
Completed Batch 10
Completed Batch 11
Completed Batch 12
Completed Batch 13
Completed Batch 14
Completed Batch 15
Completed Batch 16
Completed Batch 17
Completed Batch 18
Completed Batch 19
Completed Batch 20
Comp

In [8]:
torch.save(net.state_dict(), "/Volumes/DataVolA/ML Datasets/state-farm-distracted-driver-detection/model.pt")

In [49]:
net.load_state_dict(torch.load("/Volumes/DataVolA/ML Datasets/state-farm-distracted-driver-detection/model.pt"))

<All keys matched successfully>

In [61]:
class TestDataset(Dataset):
    def __init__(self):
        super(TestDataset, self).__init__()
        self.TEST_BASEPATH = "/Volumes/DataVolA/ML Datasets/state-farm-distracted-driver-detection/imgs/test"
        self.ts = transforms.Compose([transforms.Resize((200,200)), 
                    transforms.ToTensor(), 
                    transforms.Normalize((0.485, 0.456, 0.406), 
                                         (0.229, 0.224, 0.225))])
        self.dir_contents = [file_name for file_name in os.listdir(self.TEST_BASEPATH) if file_name!='.DS_Store']
    
    def __len__(self):
        return len(self.dir_contents)
    
    def __getitem__(self, idx):      
            img_file = self.dir_contents[idx]
            img_file_path = os.path.join(self.TEST_BASEPATH, img_file)
            img = Image.open(img_file_path)
            img_tensor = ts(img)
            return img_tensor, img_file
        
test_dataset = TestDataset()
test_loader = DataLoader(test_dataset, batch_size=200)
    

In [62]:
result_table = []
print("Started")
batch = 0
for img_tensor_batch, img_filename_batch in iter(test_loader):
    net.eval()
    batch+=1
    result = net(img_tensor_batch.to(device=device)).cpu()
    for i in range(len(img_tensor_batch)):
        result_table.append([img_filename_batch[i]]+list(result[i].detach().numpy()))
    print("--- Completed Batch {} ---".format(batch))
result_frame = pd.DataFrame(columns=['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'], 
                            data = result_table)

Started
--- Completed Batch 1 ---
--- Completed Batch 2 ---
--- Completed Batch 3 ---
--- Completed Batch 4 ---
--- Completed Batch 5 ---
--- Completed Batch 6 ---
--- Completed Batch 7 ---
--- Completed Batch 8 ---
--- Completed Batch 9 ---
--- Completed Batch 10 ---
--- Completed Batch 11 ---
--- Completed Batch 12 ---
--- Completed Batch 13 ---
--- Completed Batch 14 ---
--- Completed Batch 15 ---
--- Completed Batch 16 ---
--- Completed Batch 17 ---
--- Completed Batch 18 ---
--- Completed Batch 19 ---
--- Completed Batch 20 ---
--- Completed Batch 21 ---
--- Completed Batch 22 ---
--- Completed Batch 23 ---
--- Completed Batch 24 ---
--- Completed Batch 25 ---
--- Completed Batch 26 ---
--- Completed Batch 27 ---
--- Completed Batch 28 ---
--- Completed Batch 29 ---
--- Completed Batch 30 ---
--- Completed Batch 31 ---
--- Completed Batch 32 ---
--- Completed Batch 33 ---
--- Completed Batch 34 ---
--- Completed Batch 35 ---
--- Completed Batch 36 ---
--- Completed Batch 37 ---
--

--- Completed Batch 298 ---
--- Completed Batch 299 ---
--- Completed Batch 300 ---
--- Completed Batch 301 ---
--- Completed Batch 302 ---
--- Completed Batch 303 ---
--- Completed Batch 304 ---
--- Completed Batch 305 ---
--- Completed Batch 306 ---
--- Completed Batch 307 ---
--- Completed Batch 308 ---
--- Completed Batch 309 ---
--- Completed Batch 310 ---
--- Completed Batch 311 ---
--- Completed Batch 312 ---
--- Completed Batch 313 ---
--- Completed Batch 314 ---
--- Completed Batch 315 ---
--- Completed Batch 316 ---
--- Completed Batch 317 ---
--- Completed Batch 318 ---
--- Completed Batch 319 ---
--- Completed Batch 320 ---
--- Completed Batch 321 ---
--- Completed Batch 322 ---
--- Completed Batch 323 ---
--- Completed Batch 324 ---
--- Completed Batch 325 ---
--- Completed Batch 326 ---
--- Completed Batch 327 ---
--- Completed Batch 328 ---
--- Completed Batch 329 ---
--- Completed Batch 330 ---
--- Completed Batch 331 ---
--- Completed Batch 332 ---
--- Completed Batch 

In [65]:
result_frame.to_csv("./submission.csv", index=False)

In [31]:
[a for a in os.listdir("/Volumes/DataVolA/ML Datasets/state-farm-distracted-driver-detection/imgs/") if a!='.DS_Store']

['test', 'train']

In [48]:
x = pd.DataFrame(columns=[ 'c0', 'c1'], data=[[1,2],[3,4]])
y = pd.Series(data = [1]+[2]*3)
y

0    1
1    2
2    2
3    2
dtype: int64